In [ ]:
import numpy
import rioxarray
import xarray
import geopandas
import pathlib
import netCDF4

# Small examples of creating an xarray from numpy arrays

In [ ]:
raster_origin = [10, 35]
raster_size = [200, 100]
resolution = 10
crs = 2193

x_coords = numpy.arange(raster_origin[0], raster_origin[0] + resolution * raster_size[0], resolution)
y_coords = numpy.arange(raster_origin[1] + resolution * raster_size[1],
                        raster_origin[1], -resolution) # following convention of GDAL where y starts at the largest
z_array = numpy.zeros((1, len(y_coords), len(x_coords))) # 1 is the band dimension

example_xarray = xarray.DataArray(z_array,
                                  coords = {'band': [1], 'x': x_coords, 'y': y_coords},
                                  dims = ['band', 'y','x'], 
                                  attrs= {'scale_factor': 1.0,
                                          'add_offset': 0.0,
                                          'long_name': 'idw',
                                          '_FillValue': numpy.nan})
example_xarray = example_xarray.rio.set_crs(crs)

In [ ]:
 example_xarray

In [ ]:
example_xarray.rio.transform()

In [ ]:
example_xarray.rio.crs

# Example from Cyprien

In [ ]:
ex_path = pathlib.Path(r"C:\Users\pearsonra\Documents\data\DEMs\example_nc_file") \
    / 'Adaptdemo_coarse.nc'

In [ ]:
with rioxarray.rioxarray.open_rasterio(ex_path, masked=True) as ex_data:
    ex_data.load()
ex_data = ex_data.copy(deep=True)

In [ ]:
ex_data.rio.crs

In [ ]:
ex_data.rio.transform()

In [ ]:
ex_data

# Creating a DataSet from numpy arrays

In [ ]:
base_path = pathlib.Path(r"C:\Users\pearsonra\Documents\data\DEMs\example_nc_file")

In [ ]:
origin = [10, 35]
size = [200, 100]
resolutions = [5, 10]
crs = 2193

for resolution in resolutions:
    x = numpy.arange(origin[0],
                     origin[0] + resolution * size[0],
                        resolution)
    y = numpy.arange(origin[1] + resolution * size[1],
                     origin[1],
                     -resolution) # following convention of GDAL where y starts at the largest
    elevation = numpy.random.rand(len(x), len(y))
    roughness = numpy.random.rand(len(x), len(y))

    example_xarray = xarray.Dataset(
        data_vars=dict(elevation=(["x", "y"], elevation),
                       roughness=(["x", "y"], roughness),),
        coords=dict(
            x=(["x"], x),
            y=(["y"], y),),
        attrs= {"scale_factor": 1.0,
                "add_offset": 0.0,
                "_FillValue": numpy.nan,
                "description": f"Resolution {resolution}",
                "crs": crs})
    '''example_xarray = example_xarray.rio.set_crs(crs)
    example_xarray['elevation'] = example_xarray.elevation.rio.set_crs(crs)
    example_xarray['roughness'] = example_xarray.roughness.rio.set_crs(crs)'''
    mode = "a" if (base_path / "grouped_data.nc").exists() else "w"
    example_xarray.to_netcdf(base_path / "grouped_data.nc",
                             group=f"/resolution_{resolution}",
                             mode=mode,
                             engine="netcdf4")
    example_xarray.to_netcdf(base_path / f"data_resolution_{resolution}.nc")

In [ ]:
with rioxarray.rioxarray.open_rasterio(base_path / "grouped_data.nc", group="/resolution_5", masked=True) as ex_data:
    ex_data.load()
ex_data = ex_data.copy(deep=True)

In [ ]:
with xarray.open_dataset(base_path / "grouped_data.nc", group="/resolution_5") as ds_5:
    ds_5.rio.set_crs(int((ds.attrs['crs'])), inplace=True);
    ds_5.roughness.plot()

In [ ]:
with xarray.open_dataset(base_path / "grouped_data.nc", group="/resolution_10") as ds_10:
    ds_10.rio.set_crs(int((ds.attrs['crs'])), inplace=True);
    ds_10.roughness.plot()
    print(ds_10)

In [ ]:
with xarray.open_dataset(base_path / "data_resolution_5.nc") as ds_5:
    ds_5.rio.set_crs(int((ds.attrs['crs'])), inplace=True);
    ds_5.roughness.plot()
    print(ds_5)